In [1]:
import pandas as pd
import numpy as np


import matplotlib.patches as mpatches
import session_config
from session_config import  collect_survey_data, feature_variables, agg_groups
from reports import make_report_objects, reports_and_forecast
from reports import admin_report, features_present, histograms_standard
from reports import ecdf_plots_standard, scatter_plot_standard
from reports import labels_for_display, make_standard_report, make_report_objects
# import userdisplay
# import geospatial
import gridforecast as gfcast
# import datetime as dt
from IPython.display import Markdown

from featureevaluator import FeatureEvaluation
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from scipy.stats import spearmanr
# from sklearn.linear_model import LinearRegression, LassoCV, TheilSenRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, VotingRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, mean_squared_error
# from sklearn.inspection import permutation_importance
# from sklearn.decomposition import PCA
# from scipy.spatial import ConvexHull
# from sklearn.exceptions import ConvergenceWarning
# import warnings

import openai
from dotenv import load_dotenv
import os
from myst_nb import glue

# import bs4
# from langchain import hub
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_openai import ChatOpenAI
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import OpenAIEmbeddings
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain.chains import create_history_aware_retriever
# from langchain_core.prompts import MessagesPlaceholder
# from langchain_chroma import Chroma
# from langchain_text_splitters import MarkdownHeaderTextSplitter
# from langchain_core.messages import AIMessage, HumanMessage
# from langchain_openai import OpenAI

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

datax = collect_survey_data()
codes = pd.read_csv('data/end_process/codes.csv').set_index('code')

# from use_cases example
ooi = ['G10',  'G30', 'G31', 'G33', 'G34', 'G35', 'G8', 'G7', 'G6', 'G5', 'G4', 'G37', 'G2', 'G27', 'G25', 'G26', 'G11']
# more refined search
tobo_snacks = ['G27', 'G30', 'G35']
# unidentified, plastic, different uses
# udi = ['Gfrags', 'Gfoams']
# industrial
indus = ['G89', 'G67', 'G112', 'G93' , 'G66','G74', 'G72', 'G87', 'G65', 'G69', 'G68', 'G43', 'G41', 'G38', 'G36', 'G19', 'G17', 'Gfrags']

# features
land_covers = ['buildings', 'forest', 'undefined', 'public-services', 'recreation', 'streets']

In [3]:
o_dates = {'start':'2020-01-01', 'end':'2021-12-31'}
prior_dates = {'start':'2015-11-15', 'end':'2019-12-31'}

canton = 'Bern'
this_feature_type = 'l'
region_type = 'Canton'

d = datax.reset_index(drop=True)

d = d[d.canton.isin(['Genève', 'Valais', 'Vaud', 'Zürich', 'Bern'])]

# make complete report
params_l = {'canton':canton, 'date_range':o_dates, 'feature_type': this_feature_type}
params_p = {'canton':canton, 'date_range':prior_dates, 'feature_type':this_feature_type}

# set the parameters for the weighted prior
# exclude records in the likelihood, set date range and feature type
lu_catalogue = d[(d.canton != canton)&(d['date'] <= o_dates['end'])&(d.feature_type == 'l')&(d.code.isin(tobo_snacks))].copy()
lu_catalogue.reset_index(inplace=True, drop=True)
catalog_surveys, catalog_features = make_report_objects(lu_catalogue)

# this is the prior data: all data collected from
# the same feature type. Lakes, rivers or parks
prior_feature = catalog_features.df_cat
prior_feature['feature_type'] = 'l'

# the prior and likelihood data from the region of interest
all_data_of_interest = d[(d['date'] >= prior_dates['start']) & (d['date'] <= o_dates['end'])&(d.feature_type == 'l')&(d.code.isin(tobo_snacks))].copy()
all_data_of_interest = all_data_of_interest[all_data_of_interest.canton == canton].copy()

all_data_of_interest['use'] = 'pers'
all_data_of_interest.reset_index(inplace=True, drop=True)

land_covers = ['buildings', 'forest', 'undefined', 'public-services', 'streets', 'orchards', 'use', 'canton', 'city', 'feature_name']

all_report, all_land_use = make_report_objects(all_data_of_interest, info_columns = ['use', 'canton', 'city', 'feature_name'])


args = {
    'likelihood': {'canton':canton, 'date_range':o_dates},
    'prior' : {'canton':canton, 'date_range':prior_dates},
    'data' : all_data_of_interest.copy(),
    'land-use-inventory' : prior_feature.copy()
}


combined_results = reports_and_forecast(args['likelihood'], args['prior'], ldata=args['data'])
standard_combined = make_standard_report(combined_results, args)


lake_report = combined_results['this_report']
lake_prior_report = combined_results['prior_report']
lake_land_use = combined_results['this_land_use']

scaled_cols = ['public-services', 'buildings', 'forest', 'undefined', 'vineyards', 'orchards', 'streets', 'recreation']
system_prompt = (
    "Transcribe the values from tables and put them in paragraph form."
    "Being carefull that each value in the table is accounted for in the"
    "paragraph. You are to do this in a narrative form. Answers must be concise."
      
    "\n\n"
    "{context}"
)

indus_code_defs = codes.loc[all_data_of_interest.code.unique(), 'en']
pro_codes = (', ').join(indus_code_defs.values)

rec_code_defs = codes.loc[tobo_snacks, 'en']
rec_codes = (', ').join(rec_code_defs.values)

use_groups = {'personal':rec_codes, 'professional':pro_codes}

model = 'gpt-4o-mini'
from roughdraft import ReportTexts
client = openai.OpenAI()
args = {
    'report':lake_report,
    'landuse_report': lake_land_use,
    'client': client,
    'start': prior_dates['start'],
    'end': o_dates['end'],
    'groups': use_groups,
    'standard_report': standard_combined,
    'name': canton,
    'region_type': "Canton"
}



achatter = ReportTexts(**args)



# Rough draft prompt

In [5]:
Markdown(achatter.cluster_analysis(scaled_cols=scaled_cols))


## Cluster analysis Bern 2015-11-15 2021-12-31


Bern: Cluster composition
    
The following are the summary results of a cluster analysis. The columns are the features that were used to make the clusters. The optimal number of clusters was
determined using the elbow method (you can check the docs for this: https://hammerdirt-analyst.github.io/feb_2024/titlepage.html). The table displays the average magnitude
of each feature in the cluster. For example if the value for forest, cluster 1 = .45 then that means that in cluster 1, the average sample was taken from a location that was
45% dedicated to forest.

Table has the following format:

1. the columns are the measured land use features
2. the index is the cluster number
3. the value is the proportion of the cluster that is attributed to that column. For example if buildings in cluster 1 = .17 it means that the average magnitude of
the buildings variable was 0.17 in cluster 1.

Convert the following table into a paragraph, reporting the values for each column along with their cluster number values without any comments or analysis:

|   cluster |   public-services |   buildings |   forest |   undefined |   vineyards |   streets |   recreation |
|----------:|------------------:|------------:|---------:|------------:|------------:|----------:|-------------:|
|         0 |             0.024 |       0.591 |    0.22  |       0.189 |       0     | 0.363896  |        0.005 |
|         1 |             0.007 |       0.159 |    0.537 |       0.245 |       0     | 0.0845193 |        0.005 |
|         2 |             0.061 |       0.309 |    0.107 |       0.541 |       0.02  | 0.49708   |        0.025 |
|         3 |             0.003 |       0.047 |    0.271 |       0.55  |       0     | 0         |        0.155 |
|         4 |             0.044 |       0.682 |    0.153 |       0.149 |       0.015 | 0.251846  |        0.005 |





Bern: Average density per cluster
    
The following are the observed sample average per cluster. The units is objects per meter of beach. The columns are the use case of the objects: personal or professional. The index is
the cluster number.

Table has the following format:

1. the columns are the object use case
2. the index is the cluster number
3. the value is the objects found per meter of beach

Convert the following table into a paragraph, reporting the values for each column along with their respective cluster values without any comments or analysis:
The narrative needs to be in paragraph format.

|   cluster |    pcs/m |
|----------:|---------:|
|         0 | 1.23192  |
|         1 | 0.147857 |
|         2 | 0.492941 |
|         3 | 0.695833 |
|         4 | 0.356    |   




In [6]:
Markdown(achatter.linear_and_ensemble_regression())


## Summary of regression methods Bern 2015-11-15 2021-12-31: The different linear models the data were tested on





The following table details the results from different regression analysis of our data. 

Table has the following format:

1. Model: the type of regression model used
2. R²: The coefficient of determination
3. MSE: the mean squared error

Generate a narrative summary based on the following table. You need to include all the models and the R² and MSE result.
The narrative needs to be in paragraph format.


|    | Model                                |       R² |      MSE |
|---:|:-------------------------------------|---------:|---------:|
|  0 | Linear Regression                    | 0.322546 | 0.420651 |
|  1 | Random Forest Regression             | 0.260129 | 0.459407 |
|  2 | Gradient Boosting Regression         | 0.440842 | 0.347197 |
|  3 | Theil-Sen Regressor                  | 0.373587 | 0.388957 |
|  4 | Bagging:Gradient Boosting Regression | 0.428094 | 0.355113 |
|  5 | Voting                               | 0.381679 | 0.383933 |




In [7]:
Markdown(achatter.feature_importance())


__Model feature importance__

Feature importance is a technique used in machine learning to identify and quantify the significance of different input variables (features) in predicting the target variable. In models like decision trees, random forests, and gradient boosting machines, feature importance is often calculated by measuring how much the model's accuracy or error changes when a particular feature is included versus when it is excluded. 

The following table details the model feature importance. 

Table has the following format:

1. Feature: the name of the land-use feature
2. importance: The model feature importance

Convert the following table into a paragraph, reporting the values for each row without any comments or analysis:

|    | Feature         |   Importance |
|---:|:----------------|-------------:|
|  5 | streets         |   0.505726   |
|  2 | forest          |   0.217602   |
|  6 | recreation      |   0.140917   |
|  0 | public-services |   0.0687852  |
|  4 | vineyards       |   0.0557621  |
|  1 | buildings       |   0.00824573 |
|  3 | undefined       |   0.00296241 |





__Permutation feature importance__

Permutation importance is a model-agnostic method for assessing the importance of individual features in a predictive model. It is particularly useful because it can be applied to any type of model, whether it's a linear model, a decision tree, or a complex ensemble model. This method involves randomly shuffling the values of a feature in the dataset and observing the impact on the model's performance. A significant drop in performance indicates that the feature is important.

The following table details the model feature importance. 

Table has the following format:

1. Feature: the name of the land-use feature
2. importance: The model feature importance

Convert the following table into a paragraph, reporting the values for each row without any comments or analysis:

|    | Feature         |   Importance |
|---:|:----------------|-------------:|
|  5 | streets         |   0.399941   |
|  2 | forest          |   0.14721    |
|  6 | recreation      |   0.0300706  |
|  0 | public-services |   0.0226841  |
|  1 | buildings       |   0.0134503  |
|  4 | vineyards       |   0          |
|  3 | undefined       |  -0.00170361 |




In [8]:
Markdown(achatter.grid_approximation())


## Forecasts

Grid approximation of the posterior is a technique used in Bayesian inference to estimate the posterior distribution of parameters by discretizing the parameter space and evaluating the posterior probability at each point on a grid. This method is especially useful when the posterior distribution does not have a closed-form solution and when the parameter space is of low dimensionality, allowing for a feasible grid search. We use two priors, one is a weigheted combination of survey results from locations similar to the likelihood. The other prior uses the samples taken from the same location, but prior to the specified date range


__Bern: Weighted prior forecast____

The following contains the expected distribution of survey results.

The table has the following format:

1. average: the expected average sample total
2. hdi min: the minimum of the 90% Highest Density Interval
3. hdi max: the maximum of the 90% of the Highest Density Interval
4. 5th, 25th, 50th, 75th, 95th : the percentile rankings based on the expected distribution
5. max predicted: the maximum value predicted by the model

Generate a narrative summary based on the following table. Include all values. Reply in paragraph format, do not comment do not embelish. Use the following style guide:

|               |   expected |
|:--------------|-----------:|
| average       |     1.3861 |
| hdi min       |     0      |
| hdi max       |     7.39   |
| 5th           |     0.01   |
| 25th          |     0.22   |
| 50th          |     0.42   |
| 75th          |     1.5    |
| 95th          |     5.87   |
| max predicted |    13.49   |





__Bern: Observed 99th percentile forecast__

The following contains the expected distribution of survey results.

The table has the following format:

1. average: the expected average sample total
2. hdi min: the minimum of the 90% Highest Density Interval
3. hdi max: the maximum of the 90% of the Highest Density Interval
4. 5th, 25th, 50th, 75th, 95th : the percentile rankings based on the expected distribution
5. max predicted: the maximum value predicted by the model

Generate a narrative summary based on the following table. Include all values. Reply in paragraph format, do not comment do not embelish. Use the following style guide:

|               |   expected |
|:--------------|-----------:|
| average       |     0.5074 |
| hdi min       |     0      |
| hdi max       |     1.76   |
| 5th           |     0      |
| 25th          |     0.1275 |
| 50th          |     0.21   |
| 75th          |     0.68   |
| 95th          |     1.76   |
| max predicted |     3.75   |




In [9]:
Markdown(achatter.landuse_profile())


## Sampling stratification Bern 2015-11-15 2021-12-31: The environmental features surrounding the survey location.

Each survey location is surounded by a buffer zone of radius = 1 500 meters. The buffer zone is comprised of land-use features, each land use feature occupies a proportion of the buffer zone (0 - 100%). The land-use-profile is measured by considering the proportion of the buffer dedicated to each of land use feature that is present in the buffer zone. Each location has the same size buffer zone. What changes is how the land use features are distributed within the buffer zone, That is we can group locations by the similarity of the measured topographical features present in the buffer





The sampling stratification quantifies what proportion of the samples were conducted according to the proportion of the buffer zone that is dedicated to a particular land use feature.
Each survey location is surrounded by a buffer zone of radius = 1 500 meters. The buffer zone is comprised of land-use features, each land use feature occupies a proportion of the buffer zone (0 - 100%). 
The sampling stratification is measured by considering the proportion of the buffer dedicated to each of land use feature that is present in the buffer zone. Each location has the same size buffer zone. 
What changes is how the land use features are distributed within the buffer zone, That is we can group locations by the similarity of the measured topographical present in the buffer zone. 

__Example of how to interpret sampling-stratification table:__




__Sample table:__



|   Proportion of buffer zone |   ('Proportion of samples collected', 'buildings') |   ('Proportion of samples collected', 'wetlands') |   ('Proportion of samples collected', 'forest') |   ('Proportion of samples collected', 'public-services') |   ('Proportion of samples collected', 'recreation') |   ('Proportion of samples collected', 'undefined') |   ('Proportion of samples collected', 'streets') |   ('Proportion of samples collected', 'vineyards') |   ('Proportion of samples collected', 'orchards') |
|----------------------------:|---------------------------------------------------:|--------------------------------------------------:|------------------------------------------------:|---------------------------------------------------------:|----------------------------------------------------:|---------------------------------------------------:|-------------------------------------------------:|---------------------------------------------------:|--------------------------------------------------:|
|                           1 |                                          0.0588235 |                                                 1 |                                       0.976471  |                                                0.776471  |                                                   1 |                                          0.870588  |                                                0 |                                          0.976471  |                                                 1 |
|                           2 |                                          0.0588235 |                                                 0 |                                       0.0235294 |                                                0.2       |                                                   0 |                                          0.0470588 |                                                0 |                                          0         |                                                 0 |
|                           3 |                                          0.0117647 |                                                 0 |                                       0         |                                                0.0235294 |                                                   0 |                                          0.0823529 |                                                0 |                                          0.0235294 |                                                 0 |
|                           4 |                                          0.376471  |                                                 0 |                                       0         |                                                0         |                                                   0 |                                          0         |                                                0 |                                          0         |                                                 0 |
|                           5 |                                          0.494118  |                                                 0 |                                       0         |                                                0         |                                                   0 |                                          0         |                                                0 |                                          0         |                                                 0 |


__exmple paragraph__

The sampling-stratification of the surveys was as follows: 49% of the surveys were conducted at locations where 80-100% of the buffer was dedicated to buidlings. 37% of the surveys were taken where 60 -80% of the buffer
was dedicated to buidlings. 1% of the surveys were taken where 40-60% of the buffer was dedicated to buidlings. 6% of the samples were taken from locations where 20 - 40% of the buffer was dedicated to buidlings. 
6% of samples was taken from locations where 0-20% of the buffer was dedicated to buidlings. All of the samples were taken in locations where 0-20% of the buffer was dedicated to wetlands. 98% of the samples were 
taken from locations where 0-20% of the buffer was dedicated to forest. 2% of surveys were taken where 20-40% of the buffer was dedicated to forest. 77% of the samples were taken from locations that had
0-20% of the buffer dedicated to public-services. 20% of the surveys were taken from locations that had 20-40% of the buffer dedicated to public services. 2% of surveys were taked from locations that had
20-40% of the buffer dedicated to public services.


Convert the following table into a paragraph use the example given above to report the values for each column and index pair.

| proportion of buffer   |   ('Proportion of samples collected', 'buildings') |   ('Proportion of samples collected', 'wetlands') |   ('Proportion of samples collected', 'forest') |   ('Proportion of samples collected', 'public-services') |   ('Proportion of samples collected', 'recreation') |   ('Proportion of samples collected', 'undefined') |   ('Proportion of samples collected', 'streets') |   ('Proportion of samples collected', 'vineyards') |   ('Proportion of samples collected', 'orchards') |
|:-----------------------|---------------------------------------------------:|--------------------------------------------------:|------------------------------------------------:|---------------------------------------------------------:|----------------------------------------------------:|---------------------------------------------------:|-------------------------------------------------:|---------------------------------------------------:|--------------------------------------------------:|
| 0-20%                  |                                          0.310811  |                                                 1 |                                       0.243243  |                                                        1 |                                                   1 |                                          0.351351  |                                        0.445946  |                                                  1 |                                                 1 |
| 20-40%                 |                                          0.378378  |                                                 0 |                                       0.662162  |                                                        0 |                                                   0 |                                          0.0810811 |                                        0.0810811 |                                                  0 |                                                 0 |
| 40-60%                 |                                          0.216216  |                                                 0 |                                       0.0945946 |                                                        0 |                                                   0 |                                          0.567568  |                                        0.189189  |                                                  0 |                                                 0 |
| 60-80%                 |                                          0.0675676 |                                                 0 |                                       0         |                                                        0 |                                                   0 |                                          0         |                                        0.202703  |                                                  0 |                                                 0 |
| 80-100%                |                                          0.027027  |                                                 0 |                                       0         |                                                        0 |                                                   0 |                                          0         |                                        0.0810811 |                                                  0 |                                                 0 |





In [10]:
Markdown(achatter.landuse_rates())


## Topography and trash density Bern 2015-11-15 2021-12-31: The changes in the observed litter density and the changes in land use






The table has the following format:

1. Index = proportion of buffer occupied by feature: (1 - 5) or (0-20%, 20-40%, 40-60%, 60-80%, 80-100%)
2. Columns = The named topographical feature that maybe in the buffer
3. Values = the average pieces of trash per meter that was observed at the magnitude and feature

Convert the following table into a paragraph, reporting the values for each column along with their respective index values without any comments or analysis:

|    |   buildings |   wetlands |   forest |   public-services |   recreation |   undefined |   streets |   vineyards |   orchards |
|---:|------------:|-----------:|---------:|------------------:|-------------:|------------:|----------:|------------:|-----------:|
|  1 |    0.608261 |   0.710946 | 0.359444 |          0.710946 |     0.710946 |    1.14731  |  0.350606 |    0.710946 |   0.710946 |
|  2 |    0.235714 |   0        | 0.907959 |          0        |     0        |    1.175    |  0.681667 |    0        |   0        |
|  3 |    1.83188  |   0        | 0.235714 |          0        |     0        |    0.374524 |  0.344286 |    0        |   0        |
|  4 |    0.414    |   0        | 0        |          0        |     0        |    0        |  1.942    |    0        |   0        |
|  5 |    0.32     |   0        | 0        |          0        |     0        |    0        |  0.5      |    0        |   0        |




In [11]:
Markdown(achatter.material_composition())


## Material composition of objects Bern 2015-11-15 2021-12-31: estimated material composition





The following table details the proportion that each material type represents to the total

Generate a narrative summary based on the following table. You need to include all the material types and their float values


| material   | % of total   |
|:-----------|:-------------|
| plastic    | 100%         |




In [12]:
Markdown(achatter.summary_statistics())


## Summary statistics Bern 2015-11-15 2021-12-31: The descriptive statistics of the survey results




These are the survey totals for the data we are studying. We are analyzing count data from beach-litter surveys. The table has the following format:

1. total (quantity) = the total number of objects identified
2. nsamples = the numner of samples collected
3. average = average objects per meter

4. 5th, 25th, 50th,	75th, 95th = the objects per meter percentile ranking
5. std = standard deviation in objects per meter
6. max = the maximum recorded objects per meter
7. start = the date of the first sample
8. end = the date of the las sample

Generate a narrative summary based on the following table.

|        |   total |   nsamples |   average |   5th |   25th |   50th |   75th |   95th |      std |   max | start               | end                 |
|:-------|--------:|-----------:|----------:|------:|-------:|-------:|-------:|-------:|---------:|------:|:--------------------|:--------------------|
| result |    2180 |         74 |  0.710946 |  0.02 |  0.145 |  0.355 |  0.745 |  2.598 | 0.912234 |  4.04 | 2020-01-26 00:00:00 | 2021-04-08 00:00:00 |




In [13]:
Markdown(achatter.survey_totals_boundary(info_columns=['city']))


## Municipal results Bern 2015-11-15 2021-12-31 : The average pcs/m by municipality.





The following table details the quantity (total number of objects) and the average pieces per meter or number of objects per meter for each municipality.

Generate a narrative summary based on the following table. You need to include all the cities and their results


| city        |   quantity |     pcs/m |
|:------------|-----------:|----------:|
| Beatenberg  |         29 | 0.665     |
| Biel/Bienne |       1028 | 1.83667   |
| Brienz (BE) |         70 | 0.49      |
| Bönigen     |        107 | 1.255     |
| Erlach      |         29 | 0.51      |
| Gals        |          9 | 0.24      |
| Ligerz      |         25 | 1.605     |
| Lüscherz    |         18 | 0.064     |
| Nidau       |         15 | 0.6       |
| Spiez       |         51 | 0.0814286 |
| Thun        |         67 | 0.32      |
| Unterseen   |        669 | 0.695833  |
| Vinelz      |         63 | 0.348333  |




In [14]:
Markdown(achatter.the_admin_boundaries())


## Administrative boundaries Bern 2015-11-15 2021-12-31 : Cities, cantons, survey areas and survey locations




The following table details the number of survey locations, cities, cantons and survey areas present in the data under analysis. 

Please provide a concise narrative of the contents of the following table. In your narrative be sure to include the list of cities, 
and the names of the canton and survey areas.

 |              |   count |
|:-------------|--------:|
| location     |      19 |
| city         |      13 |
| canton       |       1 |
| survey areas |       1 |

The following is the names of the cities, cantons, and survey areas.

{'location': array(['weissenau-neuhaus', 'mullermatte', 'bielersee_vinelz_fankhausers',
       'thun-strandbad', 'oben-am-see', 'thunersee_spiez_meierd_1',
       'luscherz-plage', 'strandboden-biel', 'nidau-strand', 'delta-park',
       'sundbach-strand', 'wycheley', 'camping-gwatt-strand',
       'augustmutzenbergstrandweg', 'ligerz-strand', 'hafeli',
       'gals-reserve', 'luscherz-two', 'erlach-camping-strand'],
      dtype=object), 'city': array(['Unterseen', 'Biel/Bienne', 'Vinelz', 'Thun', 'Brienz (BE)',
       'Spiez', 'Lüscherz', 'Nidau', 'Beatenberg', 'Ligerz', 'Bönigen',
       'Gals', 'Erlach'], dtype=object), 'canton': array(['Bern'], dtype=object), 'survey_area': array(['aare'], dtype=object)}    


In [15]:
Markdown(achatter.the_named_features())


## Named features Bern 2015-11-15 2021-12-31 : The lakes, rivers and parks




The following table details the number and the name of the lakes, rivers and parks in the survey data under analysis. 

Please provide a concise narrative of the contents of the following table. In your narrative be sure to the name of each park, lake or river
that is mentioned.

|       |   lake |
|:------|-------:|
| count |      3 |


The following is the names of the lakes, rivers and parks included in the data.

{'r': array([], dtype=object), 'l': array(['thunersee', 'bielersee', 'brienzersee'], dtype=object), 'p': array([], dtype=object)}




In [16]:
file_name = 'report_results.md'

achatter.chat = True
achatter.chat_rep(scaled_cols, file_name, system_prompt=system_prompt)

file saved as report_results.md


In [19]:
o_dates = {'start':'2020-01-01', 'end':'2021-12-31'}
prior_dates = {'start':'2015-11-15', 'end':'2019-12-31'}

canton = 'Vaud'
this_feature_type = 'l'
region_type = 'canton'

d = datax.reset_index(drop=True)

d = d[d.canton.isin(['Genève', 'Valais', 'Vaud', 'Zürich', 'Bern'])]

# make complete report
params_l = {'canton':canton, 'date_range':o_dates, 'feature_type': this_feature_type}
params_p = {'canton':canton, 'date_range':prior_dates, 'feature_type':this_feature_type}

# set the parameters for the weighted prior
# exclude records in the likelihood, set date range and feature type
lu_catalogue = d[(d.canton != canton)&(d['date'] <= o_dates['end'])&(d.feature_type == 'l')&(d.code.isin(tobo_snacks))].copy()
lu_catalogue.reset_index(inplace=True, drop=True)
catalog_surveys, catalog_features = make_report_objects(lu_catalogue)

# this is the prior data: all data collected from
# the same feature type. Lakes, rivers or parks
prior_feature = catalog_features.df_cat
prior_feature['feature_type'] = 'l'

# the prior and likelihood data from the region of interest
all_data_of_interest = d[(d['date'] >= prior_dates['start']) & (d['date'] <= o_dates['end'])&(d.feature_type == 'l')&(d.code.isin(tobo_snacks))].copy()
all_data_of_interest = all_data_of_interest[all_data_of_interest.canton == canton].copy()

all_data_of_interest['use'] = 'pers'
all_data_of_interest.reset_index(inplace=True, drop=True)

land_covers = ['buildings', 'forest', 'undefined', 'public-services', 'streets', 'orchards', 'use', 'canton', 'city', 'feature_name']

all_report, all_land_use = make_report_objects(all_data_of_interest, info_columns = ['use', 'canton', 'city', 'feature_name'])


args = {
    'likelihood': {'canton':canton, 'date_range':o_dates},
    'prior' : {'canton':canton, 'date_range':prior_dates},
    'data' : all_data_of_interest.copy(),
    'land-use-inventory' : prior_feature.copy()
}


combined_results = reports_and_forecast(args['likelihood'], args['prior'], ldata=args['data'])
standard_combined = make_standard_report(combined_results, args)


lake_report = combined_results['this_report']
lake_prior_report = combined_results['prior_report']
lake_land_use = combined_results['this_land_use']

args = {
    'report':lake_report,
    'landuse_report': lake_land_use,
    'client': client,
    'start': prior_dates['start'],
    'end': o_dates['end'],
    'groups': use_groups,
    'standard_report': standard_combined,
    'name': canton,
    'region_type':region_type
}



achatter = ReportTexts(**args)
# title = f"\n# Survey report {achatter.name} {achatter.start} {achatter.end}\n\n"
# objects = f"\n__Objects in data__\n\n{', '.join([x for x in achatter.groups.values()])}\n\n"
file_name = 'vaud_report_results.md'
achatter.chat = True
achatter.chat_rep(scaled_cols, file_name, system_prompt=system_prompt)



file saved as vaud_report_results.md
